In [ ]:
import time
mega_start = time.time()

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import joblib 
import lightgbm as lgb

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
STORE_ID_REAL = ["CA_1"]

In [ ]:
store_list = ['CA_4', 'CA_2', 'CA_3', 'CA_1', 'TX_1']

#'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']

In [ ]:
submission = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sample_submission.csv")

In [ ]:
cal = pd.read_csv("/kaggle/input/m5-processeddata/calendar_factorized.csv")
cal = cal.drop(['date', 'weekday'], axis = 1)
cal['d'] = cal['d'].str.replace("d_", "").astype(np.int16)
cal = cal[cal['d'] > 1913]
cal.head()

In [ ]:
prices = pd.read_csv("/kaggle/input/m5-processeddata/sell_prices_factorized.csv")
prices = prices[prices['wm_yr_wk'].isin(cal['wm_yr_wk'].unique())]
prices.head()

In [ ]:
test = cal.merge(prices, on = ['wm_yr_wk'])

In [ ]:
test = test.drop(['wm_yr_wk'], axis = 1)

In [ ]:
for i in test.columns:
    column = test[i]
    if column.dtype == int:
        if np.max(column) > 127:
            test[i] = test[i].astype(np.int16)
        else:
            test[i] = test[i].astype(np.int8)

In [ ]:
# SANITY CHECK
test.shape[0] == (submission.shape[0] * (submission.shape[1] - 1))

# ------------------------------------------------------------------------------------------------

In [ ]:
items = pd.read_csv("/kaggle/input/m5-processeddata/sales_train_validation_factorized.csv", usecols = [1, 2, 3])
items = pd.concat([items['item_id'], 
                   pd.get_dummies(items['dept_id'], prefix = "DEPT"),
                   pd.get_dummies(items['cat_id'], prefix = "CAT")],
                  axis = 1)

items = items.drop_duplicates()
items.head()

In [ ]:
test = test.merge(items, on = ['item_id'])
test.head()

In [ ]:
for i in test.columns:
    column = test[i]
    if column.dtype == int:
        if np.max(column) > 127:
            test[i] = test[i].astype(np.int16)
        else:
            test[i] = test[i].astype(np.int8)

In [ ]:
# SANITY CHECK
test.shape[0] == (submission.shape[0] * (submission.shape[1] - 1))

In [ ]:
test['d_sin_year'] = np.sin(2*np.pi*test['d']/365)
test['d_cos_year'] = np.cos(2*np.pi*test['d']/365)
test['d_sin_dow'] = np.sin(2*np.pi*test['d']/7)
test['d_cos_dow'] = np.cos(2*np.pi*test['d']/7)
test['d_sin_month'] = np.sin(2*np.pi*test['d']/28)
test['d_cos_month'] = np.cos(2*np.pi*test['d']/28)

In [ ]:
import gc
del items, prices, cal
gc.collect()

In [ ]:
import json
with open('/kaggle/input/m5-processeddata/item_id_dict.json') as json_file:
    item_dict = json.load(json_file)

In [ ]:
submission['id'] = submission.id.str.replace("_validation", "").str.replace("_evaluation", "")
submission['store_id'] = submission['id'].str[-4:]
submission['item_id'] = submission['id'].str[:-5]
submission['item_id'] = submission['item_id'].map(item_dict)

In [ ]:
import time

In [ ]:
submit_df_dict = dict()

In [ ]:
start_time1 = time.time()

for i in store_list: 
    
    state = i.lower()[:2]
    
    dummy = pd.read_feather("/kaggle/input/m5-dataextracttransform-partb-{}/{}_v2.feather".format(state, i))
    dummy = dummy.sort_values(['item_id', 'd'])
    dummy = dummy.groupby('item_id').tail(57)
    
    for j in dummy.columns:
        column = dummy[j]
        if column.dtype == int:
            if np.max(column) > 127:
                dummy[j] = dummy[j].astype(np.int16)
            else:
                dummy[j] = dummy[j].astype(np.int8)
        
    test_temp = test[test['store_id'] == i]
    
    store_id_lower = "{}".format(i).replace("_","").lower()
    
    estimator = joblib.load("/kaggle/input/m5-train-{}/{}_v9.pkl".format(store_id_lower, i))
    
    add_to_test = list(set(dummy.columns) - set(test_temp.columns))
    extra_test = list(set(test_temp.columns) - set(dummy.columns))
                
    test_temp = test_temp.drop(extra_test, axis = 1)
    
    for j in add_to_test:
        test_temp[j] = 0
                
    test_temp = pd.concat([dummy, test_temp[dummy.columns]], axis = 0)
    
    test_temp = test_temp.sort_values(['item_id', 'd'])

    # overwrite_col = ['item_id', 'd', 'label', 'avg2', 'avg3', 'avg5', 'avg7', 'avg14', 'avg20','avg28','avg45','avg56']
    
    for j in range(1, 57):
        
        if j < 29:

            # start_time = time.time()
            # 
            rel_rows = (test_temp['d'] >= (1913-56) + j) & (test_temp['d'] <= 1913 + j)
            rel_rows2 = test_temp['d'] == 1913 + j
            #         
            test_temp.loc[rel_rows2, 'avg2'] = test_temp[rel_rows].groupby('item_id')[['label']].rolling(2).mean().groupby('item_id').shift(1).groupby('item_id').tail(1).values
            test_temp.loc[rel_rows2, 'avg3'] = test_temp[rel_rows].groupby('item_id')[['label']].rolling(3).mean().groupby('item_id').shift(1).groupby('item_id').tail(1).values
            test_temp.loc[rel_rows2, 'avg5'] = test_temp[rel_rows].groupby('item_id')[['label']].rolling(5).mean().groupby('item_id').shift(1).groupby('item_id').tail(1).values
            test_temp.loc[rel_rows2, 'avg7'] = test_temp[rel_rows].groupby('item_id')[['label']].rolling(7).mean().groupby('item_id').shift(1).groupby('item_id').tail(1).values
            test_temp.loc[rel_rows2, 'avg14'] = test_temp[rel_rows].groupby('item_id')[['label']].rolling(14).mean().groupby('item_id').shift(1).groupby('item_id').tail(1).values
            test_temp.loc[rel_rows2, 'avg20'] = test_temp[rel_rows].groupby('item_id')[['label']].rolling(20).mean().groupby('item_id').shift(1).groupby('item_id').tail(1).values
            test_temp.loc[rel_rows2, 'avg28'] = test_temp[rel_rows].groupby('item_id')[['label']].rolling(28).mean().groupby('item_id').shift(1).groupby('item_id').tail(1).values
            test_temp.loc[rel_rows2, 'avg35'] = test_temp[rel_rows].groupby('item_id')[['label']].rolling(35).mean().groupby('item_id').shift(1).groupby('item_id').tail(1).values
            test_temp.loc[rel_rows2, 'avg45'] = test_temp[rel_rows].groupby('item_id')[['label']].rolling(45).mean().groupby('item_id').shift(1).groupby('item_id').tail(1).values
            test_temp.loc[rel_rows2, 'avg56'] = test_temp[rel_rows].groupby('item_id')[['label']].rolling(56).mean().groupby('item_id').shift(1).groupby('item_id').tail(1).values
            test_temp.loc[rel_rows2, 'avg112'] = test_temp[rel_rows].groupby('item_id')[['label']].rolling(112).mean().groupby('item_id').shift(1).groupby('item_id').tail(1).values
            test_temp.loc[rel_rows2, 'avg156'] = test_temp[rel_rows].groupby('item_id')[['label']].rolling(156).mean().groupby('item_id').shift(1).groupby('item_id').tail(1).values

            # test_temp.loc[rel_rows2, 'std2'] = test_temp[rel_rows].groupby('item_id')[['label']].rolling(2).std().groupby('item_id').shift(1).groupby('item_id').tail(1).values
            # test_temp.loc[rel_rows2, 'std3'] = test_temp[rel_rows].groupby('item_id')[['label']].rolling(3).std().groupby('item_id').shift(1).groupby('item_id').tail(1).values
            # test_temp.loc[rel_rows2, 'std5'] = test_temp[rel_rows].groupby('item_id')[['label']].rolling(5).std().groupby('item_id').shift(1).groupby('item_id').tail(1).values
            # test_temp.loc[rel_rows2, 'std7'] = test_temp[rel_rows].groupby('item_id')[['label']].rolling(7).std().groupby('item_id').shift(1).groupby('item_id').tail(1).values
            # test_temp.loc[rel_rows2, 'std14'] = test_temp[rel_rows].groupby('item_id')[['label']].rolling(14).std().groupby('item_id').shift(1).groupby('item_id').tail(1).values
            # test_temp.loc[rel_rows2, 'std20'] = test_temp[rel_rows].groupby('item_id')[['label']].rolling(20).std().groupby('item_id').shift(1).groupby('item_id').tail(1).values
            # test_temp.loc[rel_rows2, 'std28'] = test_temp[rel_rows].groupby('item_id')[['label']].rolling(28).std().groupby('item_id').shift(1).groupby('item_id').tail(1).values
            # test_temp.loc[rel_rows2, 'std35'] = test_temp[rel_rows].groupby('item_id')[['label']].rolling(35).std().groupby('item_id').shift(1).groupby('item_id').tail(1).values
            # test_temp.loc[rel_rows2, 'std45'] = test_temp[rel_rows].groupby('item_id')[['label']].rolling(45).std().groupby('item_id').shift(1).groupby('item_id').tail(1).values
            # test_temp.loc[rel_rows2, 'std56'] = test_temp[rel_rows].groupby('item_id')[['label']].rolling(56).std().groupby('item_id').shift(1).groupby('item_id').tail(1).values
    # 
            # test_temp.loc[rel_rows2, 'zero_avg3'] = test_temp.loc[rel_rows2, 'zero_avg3'].fillna(method = 'ffill').fillna(0)
            # test_temp.loc[rel_rows2, 'zero_avg7'] = test_temp.loc[rel_rows2, 'zero_avg7'].fillna(method = 'ffill').fillna(0)
            # test_temp.loc[rel_rows2, 'zero_avg14'] = test_temp.loc[rel_rows2, 'zero_avg14'].fillna(method = 'ffill').fillna(0)
            # test_temp.loc[rel_rows2, 'zero_avg28'] = test_temp.loc[rel_rows2, 'zero_avg28'].fillna(method = 'ffill').fillna(0)
        
            test_temp.loc[rel_rows2, 'label'] = estimator.predict(test_temp[rel_rows2].drop('label', axis = 1))
            
            #total_time = time.time() - start_time
            #print(i, j, total_time)
    
    test_temp['pred'] = test_temp['label']
    
    submission_1 = submission.iloc[:30490, :]
    submission_2 = submission.iloc[30490:, :]
    
    test_temp = test_temp[['item_id', 'd', 'pred']]
    
    test_temp_1 = test_temp[(test_temp['d'] >= 1914) & (test_temp['d'] < (1914 + 28))]
    test_temp_1 = test_temp_1.pivot(index='item_id',columns='d')
    submission_1 = submission_1[submission_1['store_id'] == i]
    
    submission_1.iloc[:, 1:-2] = test_temp_1.values
    
    test_temp_2 = test_temp[(test_temp['d'] >= (1914 + 28)) & (test_temp['d'] <= (1914 + 56))]
    test_temp_2 = test_temp_2.pivot(index='item_id',columns='d')
    submission_2 = submission_2[submission_2['store_id'] == i]
    
    
    submission_2.iloc[:, 1:-2] = test_temp_2.values
    
    submit_df = pd.concat([submission_1, submission_2], axis = 0).drop(['store_id', 'item_id'], axis = 1)
    
    submit_df.iloc[:3049,0] = submit_df.iloc[:3049,0] + "_validation"
    submit_df.iloc[3049:,0] = submit_df.iloc[3049:,0] + "_evaluation"
    
    del test_temp, test_temp_1, test_temp_2, submission_1, submission_2, dummy
    gc.collect()
    
    submit_df_dict[i] = submit_df
    # submit_df.to_csv("submit_{}.csv".format(i))

#print(time.time() - start_time1)

In [ ]:
submission2 = submit_df_dict['CA_1']

for i in ['CA_2', 'CA_3', 'CA_4', 'TX_1']:

# ['TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']

    submission2 = pd.concat([submission2, submit_df_dict[i]], axis = 0)

In [ ]:
submission = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sample_submission.csv")
submission = submission[['id']].merge(submission2, on = 'id', how = 'left')
submission = submission.fillna(0)
submission.to_csv("submission_A.csv", index = False)

In [ ]:
print(time.time() - mega_start)